In [68]:
from IPython.display import display, HTML
display(HTML(f"""
<div style="width:600px; overflow-x:auto; border:1px solid black;">

</div>
"""))

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [10]:
spark = (SparkSession.builder.appName("SparkSQL").master("local[4]")
         .config("spark.dynamicAllocation.enabled", "false")
         .config("spark.sql adaptive.enabled", "false")
         .getOrCreate())
sc = spark.sparkContext
spark

In [12]:
yellowTaxiSchema = (StructType([
    StructField("VendorId",IntegerType(),True),
    StructField("tpep_pickup_datetime",TimestampType(),True),
    StructField("tpep_dropoff_datetime",TimestampType(),True),
    StructField("passenger_count",DoubleType(),True),
    StructField("trip_distance",DoubleType(),True),
    StructField("RatecodeID",DoubleType(),True),
    StructField("store_and_fwd_flag",StringType(),True),
    StructField("PULocationID",IntegerType(),True),
    StructField("DOLocationID",IntegerType(),True),
    StructField("payment_type",IntegerType(),True),
    StructField("fare_amount",DoubleType(),True),
    StructField("extra",DoubleType(),True),
    StructField("mta_tax",DoubleType(),True),
    StructField("tip_amount",DoubleType(),True),
    StructField("tolls_amount",DoubleType(),True),
    StructField("improvement_surcharge",DoubleType(),True),
    StructField("total_amount",DoubleType(),True),
    StructField("congestion_surcharge",DoubleType(),True),
    StructField("airport_fee",DoubleType(),True)
]))

In [70]:
filePath = "/Users/tulasiramreddygade/Downloads/apache-spark-3-fundamentals/DataFiles/Raw/YellowTaxis_202210.csv"
yellowTaxis = (
    spark
    .read
    .option("header","true")
    .schema(yellowTaxiSchema)
    .csv(filePath)
)

yellowTaxis.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorId|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-10-01 05:33:41|  2022-10-01 05:48:39|            1.0|          1.7|       1.0|                 N|         249|         107|           1|        9.5|  3.0|    0.5|      2.6

### Create a SQL view based on python data frame

In [23]:
yellowTaxis.createOrReplaceTempView("YellowTaxis")

outputDF = spark.sql(
    "SELECT * FROM YellowTaxis"
)

outputDF.printSchema()
outputDF.count()

root
 |-- VendorId: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



3675412

### Run sql query on view

In [29]:
outputDF1 = spark.sql("SELECT * FROM YellowTaxis WHERE PULocationID = 171 ")
outputDF1.count()

49

### Read Green Taxis data and create a view

In [43]:
greenTaxisFilePath = "/Users/tulasiramreddygade/Downloads/apache-spark-3-fundamentals/DataFiles/Raw/GreenTaxis_202210.csv"

greenTaxisDF = (
    spark
    .read
    .option("header","true")
    .option("delimiter","\t")
    .schema(yellowTaxiSchema)
    .csv(greenTaxisFilePath)
)
greenTaxisDF.printSchema()
greenTaxisDF.createOrReplaceTempView("GreenTaxis")

root
 |-- VendorId: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### SQL query to merge yellow and green taxis data

In [45]:
output_df = spark.sql(
    """
    SELECT "Yellow",PULocationID,DOLocationID FROM YellowTaxis

    UNION ALL

    SELECT "Green",PULocationID,DOLocationID FROM GreenTaxis
    
    """
)

output_df.count()

3744734

### Read Taxizones data and create global temp view

In [56]:
TaxiZonesFilePath = "/Users/tulasiramreddygade/Downloads/apache-spark-3-fundamentals/DataFiles/Raw/TaxiZones.csv"

TaxiZonesSchema = "LocationID INT, Borough STRING, Zone STRING, ServiceZone STRING"

TaxiZones = (
    spark
    .read
    .schema(TaxiZonesSchema)
    .csv(TaxiZonesFilePath)
)

TaxiZones.show()

TaxiZones.createOrReplaceGlobalTempView("TaxiZones")

+----------+-------------+--------------------+-----------+
|LocationID|      Borough|                Zone|ServiceZone|
+----------+-------------+--------------------+-----------+
|         1|          EWR|      Newark Airport|        EWR|
|         2|       Queens|         Jamaica Bay|  Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|  Boro Zone|
|         4|    Manhattan|       Alphabet City|Yellow Zone|
|         5|Staten Island|       Arden Heights|  Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|  Boro Zone|
|         7|       Queens|             Astoria|  Boro Zone|
|         8|       Queens|        Astoria Park|  Boro Zone|
|         9|       Queens|          Auburndale|  Boro Zone|
|        10|       Queens|        Baisley Park|  Boro Zone|
|        11|     Brooklyn|          Bath Beach|  Boro Zone|
|        12|    Manhattan|        Battery Park|Yellow Zone|
|        13|    Manhattan|   Battery Park City|Yellow Zone|
|        14|     Brooklyn|           Bay

### Create a report
Find out number of riders grouped by Borough and TaxiType

In [63]:
spark.sql("""
    SELECT Borough, TaxiType, COUNT(*) AS TotalTrips
    FROM global_temp.TaxiZones AS TaxiZones
    LEFT JOIN
    (
            SELECT "Yellow" AS TaxiType,PULocationID FROM YellowTaxis

            UNION ALL

            SELECT "Green" AS TaxiType,PULocationID FROM GreenTaxis
    ) AllTaxis

    ON AllTaxis.PULocationID = TaxiZones.LocationID
    GROUP BY  Borough, TaxiType
    ORDER BY Borough, TaxiType
    
    """
).show()

+-------------+--------+----------+
|      Borough|TaxiType|TotalTrips|
+-------------+--------+----------+
|        Bronx|   Green|      1852|
|        Bronx|  Yellow|      4511|
|     Brooklyn|   Green|     11113|
|     Brooklyn|  Yellow|     28089|
|          EWR|   Green|        15|
|          EWR|  Yellow|      1157|
|    Manhattan|    NULL|         2|
|    Manhattan|   Green|     40545|
|    Manhattan|  Yellow|   3250695|
|       Queens|    NULL|         1|
|       Queens|   Green|     15377|
|       Queens|  Yellow|    333922|
|Staten Island|    NULL|         2|
|Staten Island|   Green|         8|
|Staten Island|  Yellow|       303|
|      Unknown|   Green|       412|
|      Unknown|  Yellow|     56735|
+-------------+--------+----------+

